In [3]:
import pandas as pd

In [ ]:
excel_file = "M1045_MonthlyCrimeDashboard_TNOCrimeData.xlsx" 
df = pd.read_excel(excel_file)


csv_file = "dataProcessCSVVersion.csv"
df.to_csv(csv_file, index=False, encoding='utf-8')

print(f"Excel has coveter to CSV:{csv_file}")
excel_file = "your_file.xlsx"  # 请替换为你的文件名
df = pd.read_excel(excel_file)

# 假设你的数据包含 "日期" 列，并且格式为 YYYY-MM-DD
year = 2024  # 你想提取的年份，请修改为你的目标年份

# 确保 "日期" 列是 datetime 格式
df["日期"] = pd.to_datetime(df["日期"], errors='coerce')

# 筛选指定年份的数据
df_filtered = df[df["日期"].dt.year == year]

# 保存为 CSV 文件
csv_file = f"filtered_{year}.csv"
df_filtered.to_csv(csv_file, index=False, encoding='utf-8')

print(f"已提取 {year} 年的数据并保存为 {csv_file}")


KeyboardInterrupt: 

In [ ]:
import pandas as pd

excel_file = "M1045_MonthlyCrimeDashboard_TNOCrimeData.xlsx" 

df = pd.read_excel(excel_file)
df['date']= pd.to_datetime(df['date'])

df['year'] =pd.to_datetime(df['year'])

df['month'] =pd.to_datetime(df['month'])

print(df.head())
print(df.dtypes)
print('hello world')
